## Koedds: A national real estate analysis

In [119]:
# To start we need to load the output from our spark cluster into a pandas dataframe
import pandas as pd
import numpy as np

                COUNTY ST_ABBR  avg(median_sale_price_yoy)  \
0     Childress County      TX                    0.993825   
1      Mahnomen County      MN                    0.992475   
2        Nevada County      AR                    0.989275   
3     Jefferson County      OK                    0.986519   
4     Claiborne County      MS                    0.981198   
...                ...     ...                         ...   
2763   Williams County      ND                    0.015527   
2764     Harper County      OK                    0.014398   
2765      Stark County      ND                    0.013172   
2766   Woodward County      OK                    0.007517   
2767   New York County      NY                    0.002316   

      avg(homes_sold_mom)  avg(median_sale_price_mom)  avg(homes_sold_yoy)  \
0                0.258333                    0.575106             0.220000   
1                0.151235                    0.559932             0.361728   
2                0.35

In [120]:
# Sourced from https://gist.githubusercontent.com/wavded/1250983/raw/bf7c1c08f7b1596ca10822baeb8049d7350b0a4b/stateCodeToFips.json
# We just need to define the json as a python map for future use
state_to_fips = {
  "AK": "02",
  "AL": "01",
  "AR": "05",
  "AS": "60",
  "AZ": "04",
  "CA": "06",
  "CO": "08",
  "CT": "09",
  "DC": "11",
  "DE": "10",
  "FL": "12",
  "GA": "13",
  "GU": "66",
  "HI": "15",
  "IA": "19",
  "ID": "16",
  "IL": "17",
  "IN": "18",
  "KS": "20",
  "KY": "21",
  "LA": "22",
  "MA": "25",
  "MD": "24",
  "ME": "23",
  "MI": "26",
  "MN": "27",
  "MO": "29",
  "MS": "28",
  "MT": "30",
  "NC": "37",
  "ND": "38",
  "NE": "31",
  "NH": "33",
  "NJ": "34",
  "NM": "35",
  "NV": "32",
  "NY": "36",
  "OH": "39",
  "OK": "40",
  "OR": "41",
  "PA": "42",
  "PR": "72",
  "RI": "44",
  "SC": "45",
  "SD": "46",
  "TN": "47",
  "TX": "48",
  "UT": "49",
  "VA": "51",
  "VI": "78",
  "VT": "50",
  "WA": "53",
  "WI": "55",
  "WV": "54",
  "WY": "56"
}

state_to_fips["CO"]

'08'

In [121]:
# Comes from the output of our Spark cluster rankings
growth = pd.read_csv('./scaled_county_metrics.csv')
# Comes from the CDC, available for download here: https://www.atsdr.cdc.gov/placeandhealth/svi/data_documentation_download.html
vulnerability = pd.read_csv("./SVI_2020_US_county.csv")
vulnerability


,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,EP_ASIAN,MP_ASIAN,EP_AIAN,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE
0,1,Alabama,AL,1001,Autauga,1001,"Autauga County, Alabama",594.455838,55639,0,...,1.2,0.3,0.3,0.2,0.0,0.1,2.0,0.7,0.2,0.3
1,1,Alabama,AL,1003,Baldwin,1003,"Baldwin County, Alabama",1589.835294,218289,0,...,0.9,0.2,0.6,0.2,0.0,0.1,1.9,0.4,0.3,0.2
2,1,Alabama,AL,1005,Barbour,1005,"Barbour County, Alabama",885.007618,25026,0,...,0.5,0.1,0.3,0.3,0.0,0.1,1.3,0.6,0.6,0.4
3,1,Alabama,AL,1007,Bibb,1007,"Bibb County, Alabama",622.469288,22374,0,...,0.3,0.4,0.1,0.1,0.0,0.2,0.4,0.3,0.0,0.2
4,1,Alabama,AL,1009,Blount,1009,"Blount County, Alabama",644.890375,57755,0,...,0.4,0.1,0.1,0.1,0.1,0.1,1.8,0.3,0.3,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,56,Wyoming,WY,56037,Sweetwater,56037,"Sweetwater County, Wyoming",10426.975770,43352,0,...,0.7,0.3,1.0,0.4,0.0,0.1,2.2,0.8,0.2,0.3
3139,56,Wyoming,WY,56039,Teton,56039,"Teton County, Wyoming",3996.846846,23356,0,...,1.4,0.8,0.2,0.2,0.1,0.1,1.0,0.6,0.6,0.8
3140,56,Wyoming,WY,56041,Uinta,56041,"Uinta County, Wyoming",2081.719808,20374,0,...,0.2,0.2,0.5,0.3,0.0,0.2,2.2,0.4,0.0,0.2
3141,56,Wyoming,WY,56043,Washakie,56043,"Washakie County, Wyoming",2238.678265,7933,0,...,0.0,0.4,0.4,0.4,0.0,0.4,3.2,0.5,0.4,0.4


In [122]:
# The output from our Spark cluster has the full names which usually end with 'County' or 'Parish'
# We need to remove those to get only the county name which is what is most easily available in our social vulnerability index data
growth['COUNTY'] = growth['COUNTY'].str.replace(' County', '')
growth['COUNTY'] = growth['COUNTY'].str.replace(' Parish', '')
growth

,COUNTY,ST_ABBR,avg(median_sale_price_yoy),avg(homes_sold_mom),avg(median_sale_price_mom),avg(homes_sold_yoy),avgHomesSoldTotal,avgMedianSalesTotal,avgHomesSoldTotalScaled,avgMedianSalesTotalScaled
0,Childress,TX,0.993825,0.258333,0.575106,0.220000,0.239167,0.784465,0.057962,0.000886
1,Mahnomen,MN,0.992475,0.151235,0.559932,0.361728,0.256481,0.776204,0.059320,0.000882
2,Nevada,AR,0.989275,0.354167,0.529024,0.250000,0.302083,0.759149,0.062896,0.000872
3,Jefferson,OK,0.986519,0.470988,0.198363,0.443210,0.457099,0.592441,0.075051,0.000779
4,Claiborne,MS,0.981198,0.115385,0.777019,0.096154,0.105769,0.879108,0.047502,0.000939
...,...,...,...,...,...,...,...,...,...,...
2763,Williams,ND,0.015527,0.061237,0.005281,2.648701,1.354969,0.010404,0.145458,0.000454
2764,Harper,OK,0.014398,0.272727,0.862081,0.136364,0.204545,0.438239,0.055247,0.000693
2765,Stark,ND,0.013172,0.062988,0.006702,0.222089,0.142539,0.009937,0.050385,0.000454
2766,Woodward,OK,0.007517,0.050382,0.028853,0.054012,0.052197,0.018185,0.043301,0.000458


In [123]:
# Merge our two dataframes based on their county and states
merged_df = pd.merge(vulnerability, growth, on=['COUNTY', 'ST_ABBR'], how='left')
# Note here that our calculation for our investment score involves equally weighting both the result from our Spark cluster analyzing
# the sale price changes year over year for all residential real estate and then the social vulnerability index that comes from the CDC.
# Whereas the result from our Spark cluster ranges from 0-1 with 1 being the best, the CDC data ranges from 0-1 with 1 being the worst.
# Thus we need to add our score with the inverse of the CDC score and then normalize to 1.
merged_df['invest_score'] = 1 - ((merged_df['avg(median_sale_price_yoy)'] + (1 - merged_df['RPL_THEMES'])) / 2)
merged_df

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,MP_OTHERRACE,avg(median_sale_price_yoy),avg(homes_sold_mom),avg(median_sale_price_mom),avg(homes_sold_yoy),avgHomesSoldTotal,avgMedianSalesTotal,avgHomesSoldTotalScaled,avgMedianSalesTotalScaled,invest_score
0,1,Alabama,AL,1001,Autauga,1001,"Autauga County, Alabama",594.455838,55639,0,...,0.3,0.054293,0.028490,0.009630,0.063141,0.045816,0.031961,0.042800,0.000466,0.729353
1,1,Alabama,AL,1003,Baldwin,1003,"Baldwin County, Alabama",1589.835294,218289,0,...,0.2,0.067683,0.021337,0.007084,0.076760,0.049048,0.037384,0.043054,0.000469,0.621308
2,1,Alabama,AL,1005,Barbour,1005,"Barbour County, Alabama",885.007618,25026,0,...,0.4,0.531371,0.207743,0.413809,1.147276,0.677510,0.472590,0.092335,0.000712,0.730665
3,1,Alabama,AL,1007,Bibb,1007,"Bibb County, Alabama",622.469288,22374,0,...,0.2,0.215929,0.238266,0.165354,0.198926,0.218596,0.190641,0.056349,0.000555,0.795936
4,1,Alabama,AL,1009,Blount,1009,"Blount County, Alabama",644.890375,57755,0,...,0.4,0.081765,0.034080,0.011789,0.073529,0.053804,0.046777,0.043427,0.000474,0.715968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,56,Wyoming,WY,56037,Sweetwater,56037,"Sweetwater County, Wyoming",10426.975770,43352,0,...,0.3,0.031778,0.044248,0.007975,0.765084,0.404666,0.019876,0.070940,0.000459,0.671111
3139,56,Wyoming,WY,56039,Teton,56039,"Teton County, Wyoming",3996.846846,23356,0,...,0.8,0.198910,0.055408,0.058046,0.029844,0.042626,0.128478,0.042550,0.000520,0.496495
3140,56,Wyoming,WY,56041,Uinta,56041,"Uinta County, Wyoming",2081.719808,20374,0,...,0.2,0.107457,0.056902,0.034686,0.051705,0.054304,0.071072,0.043466,0.000488,0.649471
3141,56,Wyoming,WY,56043,Washakie,56043,"Washakie County, Wyoming",2238.678265,7933,0,...,0.4,0.111514,0.205199,0.076227,0.223827,0.214513,0.093871,0.056029,0.000501,0.577293


In [125]:
# data.json is a GeoJSON file that has all of the counties in the US already specified which means we don't need to write custom code to generate that
# However, we need to put in our rankings into the file so that we are able to display them using the Monocle tool available here: https://urban-sustain.org/services/monocle/
# The easiest way to do this was to match on County name and FIPS code and then just do a regex find and replace
import re

with open('data.json', 'r') as file:
    file_contents = file.read()
    for index, row in merged_df.iterrows():
        county, state = row['COUNTY'], row['ST_ABBR']
        state_fip = state_to_fips[state]
        search = '"STATEFP10": "' + state_fip + '"................................................................................' + county + '"'
        result = re.search(search, file_contents)
        if result is not None:
            file_contents = file_contents[:result.end() + 2] + '"investment_score": ' + str((row['invest_score'] if not np.isnan(row['invest_score']) else 1)) + ', ' + file_contents[result.end() + 2:]
            print(f'{county}, {state} successfully scored')
        else:
            print(f"County: {county}, State: {state} has an issue")
    with open('augmented_data.json', 'w') as file2:
        file2.write(file_contents)
    


Autauga, AL successfully scored
Baldwin, AL successfully scored
Barbour, AL successfully scored
Bibb, AL successfully scored
Blount, AL successfully scored
Bullock, AL successfully scored
Butler, AL successfully scored
Calhoun, AL successfully scored
Chambers, AL successfully scored
Cherokee, AL successfully scored
Chilton, AL successfully scored
Choctaw, AL successfully scored
Clarke, AL successfully scored
Clay, AL successfully scored
Cleburne, AL successfully scored
Coffee, AL successfully scored
Colbert, AL successfully scored
Conecuh, AL successfully scored
Coosa, AL successfully scored
Covington, AL successfully scored
Crenshaw, AL successfully scored
Cullman, AL successfully scored
Dale, AL successfully scored
Dallas, AL successfully scored
DeKalb, AL successfully scored
Elmore, AL successfully scored
Escambia, AL successfully scored
Etowah, AL successfully scored
Fayette, AL successfully scored
Franklin, AL successfully scored
Geneva, AL successfully scored
Greene, AL successfu

In [127]:
# Calculate correlation between our investment analysis score and the social vulnerability index
# Surprisingly there is little to no correlation between the social vulnerability index and our real estate growth numbers.
# If the slight negative result was statistically more significant then that could line up with a more-risk more-reward
# train of thought but from our current results this is inconclusive.
merged_df['inverse_svi'] = 1 - merged_df['RPL_THEMES']
merged_df['inverse_svi'].corr(merged_df['avg(median_sale_price_yoy)'])

0.12639540390762516